In [32]:
import os
import pandas as pd
import json
import datetime
from tqdm import tqdm
import pickle
from collections import Counter

In [33]:
researcherPath = 'Files/Researchers'
errorPath = 'Files/ErrorResearchers'
researcherFiles = os.listdir(researcherPath)
errorFiles = os.listdir(errorPath)

df_researchers = pd.read_pickle("Files/Dataframes/researchers_allFrom2005.pkl")

In [3]:
key = "155263"
data = readFile('156000.txt', researcherPath + "/")[key]

NameError: name 'readFile' is not defined

In [ ]:
data['File']['data']

In [34]:
def findFirstName(name): #get first name from name
    firstName = name.replace('.', '').split()[0]
    
    return firstName

In [35]:
def getFirstName(searchName, API_name, aliasList): #assign searchName if first name is longer than 1 letter or no first names are longer than 1,assign API_name if first name is longer than 1 letter, assign first alias with firs name longer than 1 letter
    firstName = findFirstName(searchName)
    if len(firstName) <= 1: #if first name consists of 1 or less letters
        firstName_API = findFirstName(API_name)
        if len(firstName_API) <= 1 and aliasList: #if first name consists of 1 or less letters
            for i in aliasList:
                firstName_alias = findFirstName(i)
                if len(firstName_alias) > 1: #if alias first name is longer than 1 letter, save that name
                    assignedFirstName = firstName_alias
                    break
                else:
                    assignedFirstName = firstName
        else:
            assignedFirstName = firstName_API
    else: 
        assignedFirstName = firstName
        
    return assignedFirstName

In [36]:
def readFile(file, path): #function to read file
    with open(path + file) as f:
        data = json.load(f)
    return data

In [37]:
def getName(data): #get namme from json
    name = data['File']['data'][0]['name'] #only save first name
        
    return name

In [38]:
def getAliases(data): #get aliases from json
    aliases = data['File']['data'][0]['aliases'] #only save first name
        
    return aliases

In [47]:
def getFields(data): #get field
    fields = []
    paper_data = data['papers']
    n_papers = len(paper_data)
    for p in range(0, n_papers):
        field = paper_data[p]['fieldsOfStudy']
        if field:
            fields += paper_data[p]['fieldsOfStudy']
    if fields:
        assignedField = Counter(fields).most_common(1)[0][0] #assign most occurring field
        fieldCount = Counter(fields).most_common(1)[0][1]
    else:
        assignedField = 'Unknown'
        fieldCount = 'Unknown'
        
    return assignedField, fieldCount

In [48]:
def getPublicationData(data): #get dict with number of publications for each year
    publishCount = {}
    paper_data = data['papers']
    n_papers = len(paper_data)
    for p in range(0, n_papers):
        year = paper_data[p]['year']
        if year in publishCount:
            publishCount[year]+= 1 #add 1 to count
        else:
            publishCount[year] = 1 #create year, as set count as 1
                
    return publishCount

In [49]:
def getAuthorInfo(files, path): #get info about authors
    error = {}
    researcher_dict = {}
    publicationData = {}
    for file in tqdm(files):
        if file != '.DS_Store':
            data = readFile(file, path)
            for key in data.keys():
                researcher_data = data[key]
                searchName = researcher_data['SearchName'] #name searched on
                if researcher_data['File']['data'][0]['papers']: #if any data on papers
                    API_name = getName(researcher_data) #name returned
                    aliases = getAliases(researcher_data) #get aliases
                    #assignedName = assignName(searchName, API_name, aliases)
                    firstName = getFirstName(searchName, API_name, aliases)
                    dataCount = data[key]['File']['total'] #authorResults
                    authorData = data[key]['File']['data'][0] #only data from first author
                    publicationData[key] = getPublicationData(authorData)
                    if not len(publicationData[key]) == 1 and not publicationData[key].get(None):
                        maxPaperCount = sum(publicationData[key].values())
                        field, fieldCount = getFields(authorData)
                #firstYear = min(list(filter(None,publicationData[key].keys())))
                        firstYear = min(list(filter(None,publicationData[key].keys())))
                    #firstYear = min(publicationData[key].keys())
                        researcher_dict[key] = {'SearchName': searchName, 'API_Name': API_name, 'FirstName': firstName, 'AuthorCount': dataCount, 'MaxPaperCount': maxPaperCount, 'Field': field, 'FieldCount': fieldCount, 'FirstPaperYear': firstYear}
                    else:
                        error[key] = {'SearchName': searchName}
                else:
                    error[key] = {'SearchName': searchName}
    
    return researcher_dict, error, publicationData

In [50]:
def getErrorResearchers(files, path): #get searchName and ID of researchers that could not be found
    error = {}
    for file in tqdm(files):
        if file != '.DS_Store':
            data = readFile(file, path)
            for key in data.keys():
                searchName = data[key]
                error[key] = {'SearchName': searchName}
    return error

In [51]:
#old - delete later
#def getFirstName(df, searchNameCol, API_nameCol):
#    firstName_dict = {}
#    for i in df.index:
#        ID = df.iloc[i]['ID']
#        name1 = df[searchNameCol].iloc[i].split()
#        name2 = df[API_nameCol].iloc[i].split()
#        if name1: #if there is any originalName
#            firstName = max(name1[0], name2[0], key = len) #take longest first name of the 2
#            firstName_dict[ID] = firstName
        
#    return firstName_dict

In [52]:
def getCumulativePaperCount(confYear, ID, publicationDict):
    publishCount = 0
    publications = publicationDict[str(ID)]
    for key in publications:
        if key != None:
            if key < confYear :
                publishCount += publications[key]
            
    return publishCount

In [53]:
def getFirstName_errorResearchers(df, searchNameCol):
    firstName_dict = {}
    for i in df.index:
        ID = df.iloc[i]['ID']
        name = df[searchNameCol].iloc[i].split()
        firstName = name[0]
        firstName_dict[ID] = firstName
    
    return firstName_dict

#### Add additional information

In [54]:
researcherDict, errorDict, publicationData = getAuthorInfo(researcherFiles, researcherPath + "/") #dict with researcher info

100%|█████████████████████████████████████████| 187/187 [01:02<00:00,  3.01it/s]


In [55]:
#create df from researcherInfo
researcher_df = pd.DataFrame.from_dict(researcherDict, orient = 'index').reset_index().rename(columns = {'index': 'ID'})
researcher_df['ID'] = researcher_df.ID.astype(int) #ID as int

In [56]:
researchers = pd.merge(researcher_df, df_researchers,  on=['ID']) #add information to df

In [57]:
#delete
#firstNameDict = getFirstName(researcher_df, 'SearchName', 'API_Name')

In [58]:
#make firstNameDict to df
#firstName_df = pd.DataFrame.from_dict(firstNameDict, orient = 'index').reset_index().rename(columns = {'index': 'ID', 0: 'FirstName'})

In [59]:
#delete
#add first name
#apiResearchers = pd.merge(researchers, firstName_df,  on = ['ID'])

In [60]:
#add seniority
researchers['Seniority'] = researchers.Year - researchers.FirstPaperYear 

In [61]:
#apply function to calculate cumulative paper count for current conference year
researchers['CumulativePaperCount'] = researchers.apply(lambda x: getCumulativePaperCount(x.Year, x.ID, publicationData), axis = 1)

In [62]:
#add productivity (cumulative papercount / years in the field)
researchers['Productivity'] = researchers.CumulativePaperCount/researchers.Seniority

In [63]:
#select only first names longer than one letter
researchers_realNames = researchers[researchers.FirstName.str.len() > 1]
#researchers[researchers.FirstName.str.len() == 2]
#what about names on 2 letters
#check

In [64]:
researchers_realNames.to_pickle("Files/DataFrames/apiResearchers.pkl") 

### Researchers not found

In [168]:
errorResearchers = getErrorResearchers(errorFiles, errorPath + "/")

100%|███████████████████████████████████████| 186/186 [00:00<00:00, 2411.17it/s]


In [30]:
#create df with researchers not found or with no data from the API
errorResearchers_df = pd.DataFrame.from_dict(errorResearchers, orient = 'index').reset_index().rename(columns = {'index': 'ID'})
researchers_noData = pd.DataFrame.from_dict(errorDict, orient = 'index').reset_index().rename(columns = {'index': 'ID'})

In [31]:
#merge the dataframes
researchers_notFound_df = pd.concat([errorResearchers_df, researchers_noData])
researchers_notFound_df['ID'] = researchers_notFound_df.ID.astype(int) #ID as int

In [32]:
#add information to df for researchers not found
researchers_notFound = pd.merge(researchers_notFound_df, df_researchers,  on=['ID']) 

In [42]:
#crete dict with first name
firstNameDict_errorResearchers = getFirstName_errorResearchers(researchers_notFound, 'SearchName')

#make firstNameDict to df
firstName_errorResearchers_df = pd.DataFrame.from_dict(firstNameDict_errorResearchers, orient = 'index').reset_index().rename(columns = {'index': 'ID', 0: 'FirstName'})

In [43]:
#add first name to df
apiResearchers_notFound = pd.merge(researchers_notFound, firstName_errorResearchers_df,  on = ['ID'])

In [44]:
#select only researchers where first name does not contain "."
researchers_realNames = apiResearchers[~apiResearchers.FirstName.str.contains('\.')]

,ID,FirstName
0,100003,cea/inac/sp2m
1,100007,swathy
2,100009,thomas
3,100018,purdue
4,100042,carolin
...,...,...
44184,99972,juan
44185,99973,goran
44186,9998,xianxu
44187,99993,stephanie


In [ ]:
firstNameDict_errorResearchers = getFirstName(researchers_notFound, 'SearchName', 'API_Name')

In [202]:
print("Number of researchers with no data/not found:", len(researchers_notFound.ID.unique()))
print("Number of researchers found:", len(researchers.ID.unique()))
print("Number of researchers removed due to not knowing their first name:", len(apiResearchers[apiResearchers.FirstName.str.contains('\.')]))
print("Number of researchers removed due to not knowing their first name, after changing code:", 8264)

Number of researchers with no data/not found: 44189
Number of researchers found: 141794
Number of researchers removed due to not knowing their first name: 48214
Number of researchers removed due to not knowing their first name, after changing code: 8264


In [197]:
#save researchers not found
researchers_notFound.to_pickle("Files/DataFrames/researcherNotFound.pkl") 

In [178]:
#researchers_notFound[researchers_notFound.SearchName.str.contains(',')]
researchers_notFound[researchers_notFound.SearchName.str.contains('department .* ,|university .* ,|department .* , .* university')]

,ID,SearchName
231,10303,"liquid crystal materials research center, univ..."
925,112886,department of materials science \& engineering...
1203,116855,university of maryland electrical and computer...
1221,117106,"department of materials and interfaces, weizma..."
1882,125371,"ece dept. , university of utah, physics dept. ..."
1992,126706,"school of materials science and engineering, c..."
2090,127964,"department of physics, boston college, departm..."
2371,131151,"department of ee,national taiwan university, g..."
2564,133572,"department of phys. , hunan normal university"
4722,155722,"university of florida, physics dept. , univers..."


In [175]:
researchers_notFound.iloc[10].SearchName

'dpmc, university of geneva, 24,, department of physics and astronomy,, department of physics, cavendish laboratory,, dpartement de recherche fondamentale sur la matire condense - spsms,'

In [196]:
department = researchers_notFound[researchers_notFound.SearchName.str.contains('department')].reset_index()
department.iloc[10:30]

,index,ID,SearchName
10,227,102987,"liquid crystal institute, kent state universit..."
11,241,103170,department of materials science and engineering
12,245,103177,"sean p. cornelius research team, toronto metro..."
13,249,103238,"center for functional nanomaterials, bnl, cent..."
14,288,103897,department of electronics science and engineer...
15,290,103938,"biological and soft sciences sector, cavendish..."
16,297,104071,department of physics and astronomy department...
17,319,104422,"department of physics, university of texas at ..."
18,380,105332,"suny oswego - physics department, unl physics ..."
19,423,106015,"institute of physics, academia sinica, departm..."
